In [ ]:
USE_WIDGETS = True

def importEssentialLibs(USE_WIDGETS):
    import numpy as np
    if USE_WIDGETS:
        %matplotlib widget
    else:
        %matplotlib inline
    import matplotlib as mpl
    mpl.rc('text', usetex = True)
    mpl.rc('font', family = 'serif', size = 18)
    import matplotlib.pyplot as plt
    import scipy.signal as signal
    
    return np, mpl, plt, signal

In [ ]:
import numpy as np
import numpy.fft as fft
import matplotlib as mpl
mpl.rc('text', usetex = False)
mpl.rc('font', family  = 'serif', size = 18)
import matplotlib.pyplot as plt
import math
import scipy.signal as signal
import scipy.io.wavfile as wav
import scipy.io as sio
import IPython
from IPython.display import Markdown



In [ ]:
#ucitavanje signala ecg_corrupted.mat

fs = 360
Ts = 1/fs


ecg = sio.loadmat('dz2_signali/ecg_corrupted.mat')
x = ecg['ecg_corrupted'].squeeze()



t = np.arange(len(x))*Ts  


fig, ax = plt.subplots(1, 1, figsize = [10, 5])
plt.subplots_adjust(bottom=0.2, wspace = 0.3)

ax.plot(t, x)
ax.set_ylabel('$ECG$')
ax.set_xlabel('$t$ [s]')
ax.set_title('ecg_corrupted')





### 1.2
Definišemo funkciju __h = baselineDriftFilter(fs, fa, fp, Aa, Ap)__. 
Budući da se IIR filtri najčešće projektuju tako što se najpre isprojektuju analogne funkcije prenosa, a zatim se one diskretizuju nekom transformacijom, takav postupak ćemo primeniti i u razrešavanju ove tačke zadatka. Analogne funkcije prenosa se najčešće projektuju polazeći od neke prototipske funkcije NF tipa kod koje granična učestanost propusnog opsega postavljena na $\omega_p = 1\, \mathrm{rad/s}$ (tzv. normalizovanog NF prototipa). Iz prototipske funkcije se postupkom koji nazivamo transformacijom učestanosti dobija frekvencijska karakteristika željenog oblika. 

In [ ]:
np, mpl, plt, signal = importEssentialLibs(USE_WIDGETS)
from dosutils import zplane
import ipywidgets as widgets



def baselineDriftFilter(fs, fa, fp, Aa, Ap):
    Ts = 1/fs
    
    # Specifikacije digitalnog filtra
    Oa = fa/fs*2*np.pi
    Op = fp/fs*2*np.pi
    
    # Specifikacije analognog prototipa
    wa = 2/Ts*math.tan(Oa/2)
    wp = 2/Ts*math.tan(Op/2)
    
    # Specifikacije normalizovanog NF prototipa
    waN = 1 #rec je o cheb2ap pa korisitmo ucestanost nepropusnog opsega
    wpN=waN*wa/wp
    
    #red filtra realizovanog sa cheb2ap i normalizovani NF prototip
    k = wpN/waN 
    D = (10**(0.1*Aa)-1) / (10**(0.1*Ap)-1)
    N = math.ceil(np.arccosh(np.sqrt(D)) / (np.arccosh(1/k)))

    z, p, k = signal.cheb2ap(N, Aa)
    
    an = np.array(np.poly(p))
    bn = k*np.array(np.poly(z))
    if an.shape == ():
        an = np.array([an])
    if bn.shape == ():
        bn = np.array([bn])
        
    #Transformacija ucestanosti
    b, a = signal.lp2hp(bn, an, wa ); #prosledjeno wa jer se radi o cebisevljenu 2
    
    # Diskretizacija bilinearnom transformacijom
    bd, ad = signal.bilinear(b, a, fs)
    
    return bd, ad


#s = baselineDriftFilter(360, 0.4, 1, 30, 0.5)

### 1.3
Definišemo funkciju __powerLineNoiseFilter(fs, fc, Aa, Ap)__ kojom se projektuje IIR filtar nepropusnik opsega učestanosti, korišćenjem normalizovanog Čebisevljevog prototipa prve vrste. 


In [ ]:
np, mpl, plt, signal = importEssentialLibs(USE_WIDGETS)
from dosutils import zplane
import math as math

def powerLineNoiseFilter(fs, fc, Aa, Ap):
    Ts = 1/fs
    
    # Specifikacije digitalnog filtra
    Op1 = 2*(fc - 2)/fs*np.pi
    Op2 = 2*(fc + 2)/fs*np.pi
    Oa1 = 2*(fc - 0.5)/fs*np.pi
    Oa2 = 2*(fc + 0.5)/fs*np.pi
    
    # Specifikacije analognog prototipa
    wa1 = 2/Ts*math.tan(Oa1/2)
    wp1 = 2/Ts*math.tan(Op1/2)
    wp2 = 2/Ts*math.tan(Op2/2)
    wa2 = 2/Ts*math.tan(Oa2/2)
    
    # Specifikacije normalizovanog NF prototipa
    wpN = 1
    w0 =np.sqrt(wp1*wp2)
    B = (wp2 - wp1)*wpN
    waN = (B*wa1)/(w0**2 - wa1**2)
    
    # Specifikacije normalizovanog NF prototipa
    wpN = 1
    B = (wp2 - wp1)*wpN
    w0 =np.sqrt(wp1*wp2)
    waN = (B*wa1)/(w0**2 - wa1**2)
    
    #red filtra realizovanog sa cheb1ap i normalizovani NF prototip
    k = wpN/waN 
    D = (10**(0.1*Aa)-1) / (10**(0.1*Ap)-1)
    N = math.ceil(np.arccosh(np.sqrt(D)) / (np.arccosh(1/k)))
    
    z, p, k = signal.cheb1ap(N, Ap)
    
    an = np.array(np.poly(p))
    bn = k*np.array(np.poly(z))
    if an.shape == ():
        an = np.array([an])
    if bn.shape == ():
        bn = np.array([bn])
        
    #Transformacija ucestanosti
    b, a = signal.lp2bs(bn, an, w0, B)
    
    # Diskretizacija bilinearnom transformacijom
    bd, ad = signal.bilinear(b, a, fs)
    
    return bd, ad



### 1.4
Korišćenjem funkcije __baselineDriftFilter__ projektujemo filtar sa određenim specifikacijama, a nakon toga flitriramo učitani EKG signal.

In [ ]:
np, mpl, plt, signal = importEssentialLibs(USE_WIDGETS)

fs = 360
fa = 0.4
fp = 1
Aa = 30
Ap = 0.5

bd, ad = baselineDriftFilter(fs, fa, fp, Aa, Ap)

s = signal.lfilter(bd, ad, x)
t = np.arange(len(s))*1/fs

fig, ax = plt.subplots(1, 1, figsize = [10, 5])
plt.subplots_adjust(bottom=0.2, wspace = 0.3)

ax.plot(t, s)
ax.set_ylabel('$ECG$')
ax.set_xlabel('$t$')
ax.set_title('Filtrirani EKG signal')



### 1.5
Korišćenjem funkcije __powerLineNoiseFilter__ projektujemo filtar sa određenim specifikacijama, a nakon toga flitriramo signal iz prethodne tačke.

In [ ]:
np, mpl, plt, signal = importEssentialLibs(USE_WIDGETS)

fs = 360
fc = 60
Aa = 40
Ap = 0.5

bd, ad = powerLineNoiseFilter(fs, fc, Aa, Ap);

p = signal.lfilter(bd, ad, s)
t = np.arange(len(p))*1/fs

fig, ax = plt.subplots(1, 1, figsize = [9, 4])
plt.subplots_adjust(bottom=0.2, wspace = 0.3)

ax.plot(t, p)
ax.set_ylabel('$ECG$')
ax.set_xlabel('$t$')
ax.set_title('Filtrirani EKG signal')




### 1.6
Sada treba da korišćenjem ugrađene funkcije __iirdesign__ projektujemo filter istih specifikacija i prikažemo
amplitudske karakteristike.

In [ ]:
np, mpl, plt, signal = importEssentialLibs(USE_WIDGETS)
import numpy.fft as fft
import math
from dosutils import zplane

fs = 360
Ts = 1/fs


#Specifikacije filtra propusnika visokih ucestanosti (VF)

fa = 0.4
fp = 1
Aa = 30
Ap = 0.5


#Računanje amplitudske karakteristike filtra iz tačke 1.4

bd4, ad4 = baselineDriftFilter(fs, fa, fp, Aa, Ap)
fd4, Hd4 = signal.freqz(bd4, ad4, worN = 10000, fs = fs)
Ha4 = abs(Hd4)


# Specifikacije digitalnog filtra

Wa = fa/fs*2*np.pi
Wp = fp/fs*2*np.pi


# Računanje amplitudske karakteristike filtra preko iirdesign

wpVector = np.array([Wp])/np.pi;
waVector = np.array([Wa])/np.pi;
bd_iir, ad_iir = signal.iirdesign(wpVector, waVector, Ap, Aa, analog=False, ftype='cheby2', output='ba', fs=None)
fd_iir, Hd_iir = signal.freqz(bd_iir, ad_iir, worN = 10000, fs = fs)
Ha_iir = abs(Hd_iir)


#poredjenje amplitudskih karakterisitka

fig, ax = plt.subplots(1, 1, figsize = [10, 5])
plt.subplots_adjust(bottom=0.2, wspace = 0.3)

ax.semilogx(fd4, 20*np.log(Ha4), label = 'Funkcija iz tačke 1.4')
ax.semilogx(fd_iir, 20*np.log(Ha_iir), label = 'Funkcija dobijena pomoću iirdesign')
ax.set_ylabel('$|H(e^{j\Omega})|, |H(e^{j\Omega})|$')
ax.set_xlabel('$f$');
ax.set_title('Ampltiduske karakteristike VF filtara')
ax.legend(loc = 'lower right')


#Specifikacije filtra nepropusnika učestanosti (NO)
fc = 60
Aa = 40
Ap = 0.5


#Računanje amplitudske karakteristike filtra iz tačke 1.5

bd5, ad5 = powerLineNoiseFilter(fs, fc, Aa, Ap);
fd5, Hd5 = signal.freqz(bd5, ad5, worN = 10000, fs = fs)
Ha5 = abs(Hd5)


# Specifikacije digitalnog filtra

Op1 = 2*(fc - 2)/fs*np.pi
Op2 = 2*(fc + 2)/fs*np.pi
Oa1 = 2*(fc - 0.5)/fs*np.pi
Oa2 = 2*(fc + 0.5)/fs*np.pi


# Dizajn filtra preko iirdesign - racunanje njegove amplitudske k-ke

wpVector = np.array([Op1,Op2])/np.pi;
waVector = np.array([Oa1,Oa2])/np.pi;
bd_iir, ad_iir = signal.iirdesign(wpVector, waVector, Ap, Aa, analog=False, ftype='cheby1', output='ba', fs=None)
fd_iir, Hd_iir = signal.freqz(bd_iir, ad_iir, worN = 10000, fs = fs)
Ha_iir = abs(Hd_iir)


#Poređenje amplitudskih karakterisitka

fig, ax = plt.subplots(1, 1, figsize = [10, 5])
plt.subplots_adjust(bottom=0.2, wspace = 0.3)

ax.semilogx(fd5, 20*np.log(Ha5), label = 'Funkcija iz tačke 1.5')
ax.semilogx(fd_iir, 20*np.log(Ha_iir), label = 'Funkcija dobijena pomoću iirdesign')
ax.set_ylabel('$|H(e^{j\Omega})|, |H(e^{j\Omega})|$')
ax.set_xlabel('$f$');
ax.set_title('Ampltiduske karakteristike NO filtara')
ax.legend(loc = 'lower right')
plt.xlim([40,70])

### 1.7
U ovoj tački nam se traži da nacrtamo raspored nula i polova, kao i amplitudske i fazne karakteristike filtara iz tačaka 3 i 4 i da pokažemo da filtri zadovoljavaju zadate specifikacije.

In [ ]:
np, mpl, plt, signal = importEssentialLibs(USE_WIDGETS)
from dosutils import zplane
import math

fs = 360
Ts = 1/fs


# VF filtar

fig = plt.figure(figsize = [10,5])
plt.subplots_adjust(wspace = 0.3)

#specifikacije filtra propusnika visokih ucestanosti
fa = 0.4
fp = 1
Aa = 30
Ap = 0.5


#Filtar koji smo realizovali u tački 1.5

bd4, ad4 = baselineDriftFilter(fs, fa, fp, Aa, Ap)


#Plotovanje nula i polova

ax1 = fig.add_subplot(1,3,1, aspect=1)
zplane(bd4, ad4, ax1)
ax1.set_title('Raspored nula i polova')


# Frekvencijska karakteristika

fd4, Hd4 = signal.freqz(bd4, ad4, worN = 10000, fs = fs)


#Amplitudska karakteristika i njeno plotovanje

Ha4 = abs(Hd4)

ax2 = fig.add_subplot(1,3,2)
ax2.semilogx(fd4, 20*np.log10(Ha4))
ax2.set_title('Amplitudska karakteristika')
ax2.set_ylabel('$|H(j\omega)|$')
ax2.set_xlabel('$f$');
rXlim = 100
ax2.set_xlim([0.1, rXlim])
ax2.set_ylim([-55, 1])


#Stop band lim

ax2.add_patch(mpl.patches.Polygon([[0, -Aa ], [fa, -Aa], [fa, -Aa+2], [0, -Aa+2]], closed=True,
                                fill=False, hatch='////', color = 'red'))

# Pass band lim

ax2.add_patch(mpl.patches.Polygon([[fp, -Ap-2 ], [rXlim, -Ap-2], [rXlim, -Ap], [fp, -Ap]], closed=True,
                                fill=False, hatch='////', color = 'red'))


# Fazna karakteristika i njeno plotovanje

ax3 = fig.add_subplot(1,3,3)
ax3.semilogx(fd4, np.unwrap(np.angle(Hd4)))
ax3.set_title('Fazna karakteristika')
ax3.set_ylabel('$X(j\omega)$')
ax3.set_xlabel('$f$');
ax3.set_xlim([0.1, 100])



# NO filtar

fig = plt.figure(figsize = (15,5))
plt.subplots_adjust(wspace = 0.3)


# Specifikacije filtra nepropusnika učestanosti

fc = 60
Aa = 40
Ap = 0.5
fp1 = fc - 2
fp2 = fc + 2
fa1 = fc - 0.5
fa2 = fc + 0.5


# Filar koji smo realizovali u tački 1.5

bd5, ad5 = powerLineNoiseFilter(fs, fc, Aa, Ap)


# Plotovanje nula i polova

ax1 = fig.add_subplot(1,3,1, aspect=1)
zplane(bd5, ad5, ax1)
ax1.set_title('Raspored nula i polova')


# Amplitudska karakteristika i njeno plotovanje
Ha5 = abs(Hd5)

ax2 = fig.add_subplot(1,3,2)
ax2.semilogx(fd5, 20*np.log10(Ha5))
ax2.set_title('Amplitudska karakteristika')
ax2.set_ylabel('$|H(j\omega)|$')
ax2.set_xlabel('$f$');
rXlim = 70
ax2.set_xlim([50, rXlim])


# First pass band lim

ax2.add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [fp1, -Ap-2], [fp1, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))

# Stop band lim

ax2.add_patch(mpl.patches.Polygon([[fa1, -Aa ], [fa2, -Aa], [fa2, -Aa+2], [fa1, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))

# Second pass band lim

ax2.add_patch(mpl.patches.Polygon([[fp2, -Ap-2 ], [rXlim, -Ap-2], [rXlim, -Ap], [fp2, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))


# Fazna karakteristika i njeno plotovanje
ax3 = fig.add_subplot(1,3,3)
ax3.semilogx(fd5, np.unwrap(np.angle(Hd5)))
ax3.set_title('Fazna karakteristika')
ax3.set_ylabel('$X(j\omega)$')
ax3.set_xlabel('$f$');
ax3.set_xlim([40, 80])

In [ ]:
# u ovom delu učitavamo zvučni signal sa prisutnim šumom 

np, mpl, plt, signal = importEssentialLibs(True)
import numpy.fft as fft
import IPython
from scipy.io.wavfile import read as wavread
import IPython.display as ipd

def forceAspect(ax, aspect=1):
    im = ax.get_images()
    extent =  im[0].get_extent()
    ax.set_aspect(abs((extent[1]-extent[0])/(extent[3]-extent[2]))/aspect)

    
# Ucitavanje
fs, x = wavread('dz2_signali/sound_corrupted.wav')

ipd.Audio(x, rate = fs)

In [ ]:
#crtanje spektograma zvučnog signala

t = np.arange(len(x))/fs


fig, axs = plt.subplots(1, 1, figsize = [10,10], sharex=True)
plt.subplots_adjust(bottom=0.25, hspace = 0.4)

fMaxShow = 20000


Nwin = 1024*2;
fMaxShow = fs//2
window = signal.hamming(Nwin)
NFFT = Nwin

f, t, Sxx = signal.spectrogram(x, fs = fs, window=window, noverlap=Nwin//4, nfft=NFFT, return_onesided=True, 
                                   scaling='spectrum', mode='complex')
fMaxIndex = NFFT*fMaxShow//fs




axs.matshow(20*np.log10(abs(Sxx[:][:fMaxIndex])), extent=[min(t), max(t), min(f), fMaxShow], origin='lower', cmap = 'inferno')
forceAspect(axs, 3)

axs.set_xlabel('$t$ [s]')
axs.set_ylabel('$f$ [Hz]');

In [ ]:
np, mpl, plt, signal = importEssentialLibs(USE_WIDGETS)


# U ovom delu definišemo fju koja će nam pomoći pri projektovanju filtra metodom ograničavanja impulsnog odziva

def FIR1Filtar(fs, fp1, fa1, fa2, fp2, Aa, Ap):
    
    Wa1 = 2*np.pi*fa1/fs
    Wa2 = 2*np.pi*fa2/fs
    Wp1 = 2*np.pi*fp1/fs
    Wp2 = 2*np.pi*fp2/fs
    
    Bt = min([Wa1 - Wp1, Wp2 - Wa2])
    
    Wc1 = (Wa1 + Wp1)/2
    Wc2 = (Wa2 + Wp2)/2
    
    # Delta
    delta_p = (10**(0.05*Ap)-1)/(10**(0.05*Ap)+1)
    delta_a = 10**(-0.05*Aa)
    delta = min([delta_p, delta_a])

    # Alpha_a
    #if delta < delta_a:
    AaSpec = -20*np.log10(delta)
    #else:
        #AaSpec = Aa

    # Beta
    beta = 0;
    if AaSpec < 21:
        beta = 0
    elif AaSpec <= 50:
        beta = 0.5842*(AaSpec-21)**0.4 + 0.07886*(AaSpec-21)
    else:
        beta = 0.1102*(AaSpec-8.7)
        
    # Filter order
    if AaSpec <= 21:
        D = 0.9222
    else:
        D = (AaSpec-7.95)/14.36
    M = int(np.ceil(2*np.pi*D/Bt + 1))
    N = M - 1
    Norig = N
    
    filterFitsInSpecs = False
    first = True
    while not filterFitsInSpecs:
        w = signal.kaiser(N+1, beta)

        na = np.arange(-N/2, N/2+1, 1)
        n = np.arange(N+1)

        hID = np.zeros(N+1)
        if N % 2 == 0:
            na = np.delete(na, N//2) # remove index 0
            hID[n != N//2] = np.sin(np.pi*na)/np.pi/na + np.sin(Wc1*na)/np.pi/na - np.sin(Wc2*na)/np.pi/na
            hID[N//2] = 1 + Wc1/np.pi - Wc2/np.pi  # lim in na->0
        else:
            hID = np.sin(np.pi*na)/np.pi/na + np.sin(Wc1*na)/np.pi/na - np.sin(Wc2*na)/np.pi/na
            
        h = hID * w
        
        numPoints = 1024*12
        W, H = signal.freqz(h, 1, worN = numPoints)
        Hdb = 20*np.log10(abs(H))
        
        # Proveravamo ispunjenost gabarita i ukoliko nisu ispunjeni povećavamo N
        if first:
            first = False
            HdbOriginal = Hdb

        DeltaW = np.pi/numPoints
        kSB1 = int(np.ceil(Wa1/DeltaW))
        kPB1 = int(np.floor(Wp1/DeltaW))
        kPB2 = int(np.ceil(Wp2/DeltaW))
        kSB2 = int(np.floor(Wa2/DeltaW))

        #uslovi koji odgovaraju NO filtru
        if np.all(Hdb[:kPB1] > -Ap) and np.all(Hdb[:kPB1] < Ap) and \
            np.all(Hdb[kSB1:kSB2] < - Aa) and np.all(Hdb[kPB2:] < Ap) and np.all(Hdb[kPB2:] > -Ap) :
            filterFitsInSpecs = True 
        else:
            N += 1
            
        return N,w,h


In [ ]:
np, mpl, plt, signal= importEssentialLibs(USE_WIDGETS)

#Projektovanje filta optimizacionom metodom


def FIR2Filtar(fs, fp1, fa1, fa2, fp2, Aa, Ap):
    
    Wp1 = 2*np.pi*fp1/fs
    Wa1 = 2*np.pi*fa1/fs
    Wa2 = 2*np.pi*fa2/fs
    Wp2 = 2*np.pi*fp2/fs
    
    dp = (10**(0.05*Ap)-1)/(10**(0.05*Ap)+1)
    da = 10**(-0.05*Aa)

    
    D = (0.01201*np.log10(dp)**2 + 0.09664*np.log10(dp) - 0.51325)*np.log10(da)
    D = D + (0.00203*np.log10(dp)**2 - 0.57054*np.log10(dp) - 0.44314)
    
    #f(delta_p,delta_a)
    f = -16.9 + 14.6*(np.log10(dp) - np.log10(da))
    
    Bt = min([Wa1-Wp1,Wp2-Wa2])
    M = int(np.ceil(2*np.pi*D/Bt - f*Bt/(2*np.pi) + 1))

    # Relativne ucestanosti
    Fp1 = fp1/fs
    Fa1 = fa1/fs
    Fp2 = fp2/fs
    Fa2 = fa2/fs
    
    # Željena idealna amplitudska karakteristika
    HZ = [1, 0, 1]
    
    # Vektor relativnih učestanosti za koje se zadaje željena učestanost HZ
    F = [0, Fp1, Fa1, Fa2, Fp2, 0.5]
    
    # Vektor težinskih faktora
    W = [1, dp/da, 1] 
    
    numPoints = 10240
    filterFitsInSpecs = False
    first = True
    while not filterFitsInSpecs:
        h = signal.remez(M, F, HZ, W)
        w, H = signal.freqz(h, 1, worN =numPoints )
        Hdb = 20*np.log10(abs(H))
        
        DeltaW = np.pi/numPoints
        kSB1 = int(np.ceil(Wa1/DeltaW))
        kPB1 = int(np.floor(Wp1/DeltaW))
        kPB2 = int(np.ceil(Wp2/DeltaW))
        kSB2 = int(np.floor(Wa2/DeltaW))
        
        if np.all(Hdb[:kPB1] > -Ap) and np.all(Hdb[:kPB1] < Ap) and \
            np.all(Hdb[kSB1:kSB2] < - Aa) and np.all(Hdb[kPB2:] < Ap) and np.all(Hdb[kPB2:] > -Ap) :
            filterFitsInSpecs = True 
        else:
            M += 1
            
        return M,w,h

In [ ]:
np, mpl, plt, signal= importEssentialLibs(USE_WIDGETS)

def clear1():

    #filtriranje frekvencijske komponente oko 10000 Hz
    N3, w, h = FIR1Filtar(fs, 6400, 6500, 15000, 15600, 40, 0.5)
    c = signal.lfilter(h,1,x)

    #filtriranje frekvencijske komponente oko 2750 Hz
    N2, w, h = FIR1Filtar(fs, 2450, 2550, 2750, 2800, 40, 0.5)
    b = signal.lfilter(h,1,c)

    #filtriranje frekvencijske komponente oko 1550 Hz
    N1, w, h = FIR1Filtar(fs, 1460, 1500, 1560, 1600, 40, 0.5)
    a = signal.lfilter(h,1,b)
    
    return a, fs, N1, N2, N3

def clear2():

    #filtriranje frekvencijske komponente oko 10000 Hz
    N3, w, h = FIR2Filtar(fs, 6400, 6500, 14800, 15000, 40, 0.5)
    f = signal.lfilter(h, 1, x)

     #filtriranje frekvencijske komponente oko 2750 Hz
    N2, w, h = FIR2Filtar(fs,2450, 2550, 2750, 2800, 40, 0.5)
    e = signal.lfilter(h, 1, f)

    #filtriranje frekvencijske komponente oko 1550 Hz
    N1, w, h = FIR2Filtar(fs, 1460, 1500, 1560, 1600, 40, 0.5)
    d = signal.lfilter(h, 1, e)
    
    return d, fs, N1, N2, N3

sound1, fs, N1, N2, N3 = clear1()

print("N3 =", N3)
print("N2 =", N2)
print("N1 =", N1)
    
IPython.display.display(IPython.display.Audio(sound1, rate = fs))



In [ ]:
sound2, fs, N1, N2, N3 = clear2()

print("N3 =",N3)
print("N2 =",N2)
print("N1 =",N1)

IPython.display.display(IPython.display.Audio(sound2, rate = fs))

In [ ]:
fig, axs = plt.subplots(2, 1, figsize = [10,10], sharex=True)
plt.subplots_adjust(bottom=0.25, hspace = 0.4)



fMaxShow = 20000


Nwin = 1024*2;
fMaxShow = fs//2
window = signal.hamming(Nwin)
NFFT = Nwin

f, t, Sxx = signal.spectrogram(sound1, fs = fs, window=window, noverlap=Nwin//4, nfft=NFFT, return_onesided=True, 
                                   scaling='spectrum', mode='complex')
fMaxIndex = NFFT*fMaxShow//fs





axs[0].matshow(20*np.log10(abs(Sxx[:][:fMaxIndex])), extent=[min(t), max(t), min(f), fMaxShow], origin='lower', cmap = 'inferno')
forceAspect(axs[0], 3)

axs[0].set_title('Spektogram filtriranja metodom ograničavanja impulsnog odziva')


f, t, Sxx = signal.spectrogram(sound2, fs = fs, window=window, noverlap=Nwin//4, nfft=NFFT, return_onesided=True, 
                               scaling='spectrum', mode='complex')
fMaxIndex = NFFT*fMaxShow//fs


axs[1].matshow(20*np.log10(abs(Sxx[:][:fMaxIndex])), extent=[min(t), max(t), min(f), fMaxShow], origin='lower', cmap = 'inferno')
forceAspect(axs[1], 3)

axs[1].set_title('Spektogram filtriranja optimizacionim postupkom projektovanja')



In [ ]:
np, mpl, plt, signal= importEssentialLibs(USE_WIDGETS)

#(fp1, fa1, fa2, fp2, Aa, Ap, W, H, xMin, xMax, fs, title)


fp5 = 6400
fa5 = 6500
fa6 = 15000
fp6 = 15600

fp3 = 2450
fa3 = 2550
fa4 = 2750
fp4 = 2800


fp1 = 1460
fa1 = 1500
fa2 = 1560
fp2 = 1600



Wp1 = 2*np.pi*fp1/fs
Wa1 = 2*np.pi*fa1/fs
Wa2 = 2*np.pi*fa2/fs
Wp2 = 2*np.pi*fp2/fs
 
    
Wp3 = 2*np.pi*fp3/fs
Wa3 = 2*np.pi*fa3/fs
Wa4 = 2*np.pi*fa4/fs
Wp4 = 2*np.pi*fp4/fs

Wp5 = 2*np.pi*fp5/fs
Wa5 = 2*np.pi*fa5/fs
Wa6 = 2*np.pi*fa6/fs
Wp6 = 2*np.pi*fp6/fs
    
    
fig, ax = plt.subplots(3,1, figsize = [10, 20])
plt.subplots_adjust(bottom=0.15, wspace = 0.4)
 



Aa = 40
Ap = 1
numPoints = 10240

#filtri

#prvi
N1, w, h = FIR2Filtar(fs, 1460, 1500, 1560, 1600, 40, 0.5)
W, H = signal.freqz(h, 1 , worN = numPoints)
ax[0].plot(W,20*np.log10(abs(H)));
ax[0].set_title('NO filtar oko 1550Hz optimizacionom metodom')
ax[0].set_xlabel(r'$f$ Hz');
ax[0].set_ylabel(r'$H$[dB]');
ax[0].set_xlim([0, 0.5]);

# First pass band lim
ax[0].add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [Wp1, -Ap-2], [Wp1, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Stop band lim
ax[0].add_patch(mpl.patches.Polygon([[Wa1, -Aa ], [Wa2, -Aa], [Wa2, -Aa+2], [Wa1, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Second pass band lim
ax[0].add_patch(mpl.patches.Polygon([[Wp2, -Ap-2 ], [0.5, -Ap-2], [0.5, -Ap], [Wp2, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
    
    
#drugi
N2, w, h = FIR2Filtar(fs,2450, 2550, 2750, 2800, 40, 0.5)
W, H = signal.freqz(h, 1 , worN = numPoints)
ax[1].plot(W,20*np.log10(abs(H)));
ax[1].set_title('NO filtar oko 2750Hz optimizacionom metodom')
ax[1].set_xlabel(r'$f$ Hz');
ax[1].set_ylabel(r'$H$[dB]');
ax[1].set_xlim([0.2, 0.7]);
# First pass band lim
ax[1].add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [Wp3, -Ap-2], [Wp3, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Stop band lim
ax[1].add_patch(mpl.patches.Polygon([[Wa3, -Aa ], [Wa4, -Aa], [Wa4, -Aa+2], [Wa3, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Second pass band lim
ax[1].add_patch(mpl.patches.Polygon([[Wp4, -Ap-2 ], [0.7, -Ap-2], [0.7, -Ap], [Wp4, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))



#treci 
 
N3, w, h = FIR2Filtar(fs, 6400, 6500, 14800, 15000, 40, 0.5)
W, H = signal.freqz(h, 1 , worN = numPoints)
ax[2].plot(W,20*np.log10(abs(H)));
ax[2].set_title('NO filtar oko 10000Hz optimizacionom metodom')
ax[2].set_xlabel(r'$f$ Hz');
ax[2].set_ylabel(r'$H$[dB]');
ax[2].set_xlim([0.2, 3.5]);
# First pass band lim
ax[2].add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [Wp5, -Ap-2], [Wp5, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Stop band lim
ax[2].add_patch(mpl.patches.Polygon([[Wa5, -Aa ], [Wa6, -Aa], [Wa6, -Aa+2], [Wa5, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Second pass band lim
ax[2].add_patch(mpl.patches.Polygon([[Wp6, -Ap-2 ], [3.5, -Ap-2], [3.5, -Ap], [Wp6, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))


In [ ]:
np, mpl, plt, signal= importEssentialLibs(USE_WIDGETS)

#(fp1, fa1, fa2, fp2, Aa, Ap, W, H, xMin, xMax, fs, title)


fp5 = 6400
fa5 = 6500
fa6 = 15000
fp6 = 15600

fp3 = 2450
fa3 = 2550
fa4 = 2750
fp4 = 2800


fp1 = 1460
fa1 = 1500
fa2 = 1560
fp2 = 1600



Wp1 = 2*np.pi*fp1/fs
Wa1 = 2*np.pi*fa1/fs
Wa2 = 2*np.pi*fa2/fs
Wp2 = 2*np.pi*fp2/fs
 
    
Wp3 = 2*np.pi*fp3/fs
Wa3 = 2*np.pi*fa3/fs
Wa4 = 2*np.pi*fa4/fs
Wp4 = 2*np.pi*fp4/fs

Wp5 = 2*np.pi*fp5/fs
Wa5 = 2*np.pi*fa5/fs
Wa6 = 2*np.pi*fa6/fs
Wp6 = 2*np.pi*fp6/fs

    
    
fig, ax = plt.subplots(3,1, figsize = [10, 20])
plt.subplots_adjust(bottom=0.15, wspace = 0.4)
 



Aa = 40
Ap = 1
numPoints = 10240

#filtri

#prvi
N1, w, h = FIR1Filtar(fs, 1460, 1500, 1560, 1600, 40, 0.5)
W, H = signal.freqz(h, 1 , worN = numPoints)
ax[0].plot(W,20*np.log10(abs(H)));
ax[0].set_title('NO filtar oko 1550Hz metodom ograničavanja impulsnog odziva')
ax[0].set_xlabel(r'$f$ Hz');
ax[0].set_ylabel(r'$H$[dB]');
ax[0].set_xlim([0, 0.5]);
# First pass band lim
ax[0].add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [Wp1, -Ap-2], [Wp1, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Stop band lim
ax[0].add_patch(mpl.patches.Polygon([[Wa1, -Aa ], [Wa2, -Aa], [Wa2, -Aa+2], [Wa1, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Second pass band lim
ax[0].add_patch(mpl.patches.Polygon([[Wp2, -Ap-2 ], [0.5, -Ap-2], [0.5, -Ap], [Wp2, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
    
    
#drugi
N2, w, h = FIR1Filtar(fs,2450, 2550, 2750, 2800, 40, 0.5)
W, H = signal.freqz(h, 1 , worN = numPoints)
ax[1].plot(W,20*np.log10(abs(H)));
ax[1].set_title('NO filtar oko 2750Hz metodom ograničavanja impulsnog odziva')
ax[1].set_xlabel(r'$f$ Hz');
ax[1].set_ylabel(r'$H$[dB]');
ax[1].set_xlim([0.2, 0.7]);
# First pass band lim
ax[1].add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [Wp3, -Ap-2], [Wp3, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Stop band lim
ax[1].add_patch(mpl.patches.Polygon([[Wa3, -Aa ], [Wa4, -Aa], [Wa4, -Aa+2], [Wa3, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Second pass band lim
ax[1].add_patch(mpl.patches.Polygon([[Wp4, -Ap-2 ], [0.7, -Ap-2], [0.7, -Ap], [Wp4, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))



#treci 
 
N3, w, h = FIR1Filtar(fs, 6400, 6500, 14800, 15000, 40, 0.5)
W, H = signal.freqz(h, 1 , worN = numPoints)
ax[2].plot(W,20*np.log10(abs(H)));
ax[2].set_title('NO filtar oko 10000Hz metodom ograničavanja impulsnog odziva')
ax[2].set_xlabel(r'$f$ Hz');
ax[2].set_ylabel(r'$H$[dB]');
ax[2].set_xlim([0.2, 3.5]);
# First pass band lim
ax[2].add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [Wp5, -Ap-2], [Wp5, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Stop band lim
ax[2].add_patch(mpl.patches.Polygon([[Wa5, -Aa ], [Wa6, -Aa], [Wa6, -Aa+2], [Wa5, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Second pass band lim
ax[2].add_patch(mpl.patches.Polygon([[Wp6, -Ap-2 ], [3.5, -Ap-2], [3.5, -Ap], [Wp6, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
  





    